# Programming for Data Analytics Final Project: Wind

### Author: Aoife Flavin

Put a description of what the project is for here



Description of the dataset

In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
